# 背景

对指定个股和指数进行30分钟级别的监控，包括趋势、反转指标等，并通过语音进行报告。

本监控在数据上不依赖于zillionare-omega

In [32]:
! export HTTPS_PROXY=""
from alpha.notebook import *
import datetime
await init_notebook(use_omicron=False)

g = {
    "notified": set()
}

In [7]:
def get_pmae_threshold(code, frame_type, win):
    if code in ["000001.XSHG"]:
        return {
            "30m": {
                5: 1.51e-4,
                10: 1.51e-4
            }
        }.get(frame_type, {}).get(win, 3e-4)
    else: # 个股
        return {
            "30m": {
                5: 3e-3,
                10: 1e-3
            }
        }
    
def maline_turn(bars, win, err_threshold):
    close = bars["close"].copy()
    close /= close[0]
    
    ma = moving_average(close, win)[-7:]
    coef, pmae = polyfit(ma)
    if pmae > err_threshold:
        return None
        
    a, b, c = coef
    
    vx = 7 - round(-b/(2 * a))
    if 0 < vx <=7:
        return ma[-vx] < ma[-1], bars["frame"][-vx]
    else:
        return None
    
def notify(title, details):
    ft = details.get("frame_type")
    frame = details.get("frame")
    code = details.get("code")
    win = details.get('win')
    
    key = f"{code}{frame}{ft}{win}"
    
    if key not in g["notified"]:
        say(title)
        g["notified"].add(key)
        
def canonical_code(code):
    if "." in code:
        return code
    
    if code.startswith("6"):
        return code + ".XSHG"
    else:
        return code + ".XSHE"

In [28]:
def _desc_maline_turn(name, code, frame_type, win, flag, frame):
    desc = "向上拐头" if flag else "向下拐头"
    tm = arrow.get(frame, 'Asia/Shanghai').humanize(locale='zh')
    title = f"{name}30分钟线在{tm}发出{desc}信号"
    details = {
        "frame": frame,
        "flag": flag,
        "win": win,
        "frame_type": frame_type,
        "code": code
    }
    
def do_check(holdings:list[str]):
    code, name, frame_type = '000001.XSHG', '沪指', '30m'
    bars = jq_get_bars(code, 15, frame_type)
    
    update_graph = False
    for win in [5, 10]:
        res = maline_turn(bars, win, get_pmae_threshold(code, frame_type, win))
        if res is not None:
            flag, frame = res

            title, details = _desc_maline_turn(name, code, frame_type, win, flag, frame)

            notify(title, details)
            update_graph = True
            
    for code in holdings:
        bars = jq_get_bars(code, 15, '30m')
        for win in [5, 10]:
            res = maline_turn(bars, win, get_pmae_threshold(code, '30m', win))
            if res is None:
                continue
                
            flag, frame = res
            name = jq_get_name(code)
            title, details = _desc_maline_turn(name, code, '30m', win, flag, frame)
            notify(title, details)
            update_graph = True
            
            
    if update_graph:
        cs = Candlestick(frames={"30m": [5, 10, 20, 30, 60]})

        bars = jq_get_bars("000001.XSHG", 119, "30m")
        cs.plot_bars(bars, title=f"沪指 30M")
        
async def mon():
    import asyncio
    my_stocks = ["300849", "601179", "601106", "300212", "000591", "002340"]
    my_stocks = map(canonical_code, my_stocks)
    
    now = arrow.now()
    wakeup_time = []
    for tm in ["09:59", "10:29", "10:59", "11:29", "13:29", "13:59", "14:29", "14:56"]:
        hour, minute = map(int, tm.split(":"))
        wakeup_time.append(arrow.Arrow(now.year, now.month, now.day, hour, minute, tzinfo="Asia/Shanghai"))
        
    for tm in wakeup_time:
        if arrow.now() > tm:
            continue
            
        seconds = (tm.timestamp - arrow.now().timestamp)
        await asyncio.sleep(seconds)
        do_check(my_stocks)
        
await mon()

tm is 2021-10-13T22:15:00+08:00, waiting for 36 seconds
i'm checking
tm is 2021-10-13T22:16:00+08:00, waiting for 60 seconds
i'm checking
tm is 2021-10-13T22:18:00+08:00, waiting for 120 seconds
i'm checking
